In [1]:
%pip install clarabel
%pip install pysmps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.6 MB/s eta 0:00:00


In [2]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME = 'EE364B/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME) + '/tests')

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from LPSolver import LPSolver
import cvxpy as cp
from performanceTest import parse_mps_list, get_file_names, test_on_mps_data
from pysmps.smps_loader import load_mps
from time import time

In [ ]:
DATAFOLDER = '/content/drive/My Drive/{}'.format(FOLDERNAME) + '/tests/data/'
# cvxpy_times, ls_gpu_times, ls_cpu_times, datafiles = test_on_mps_data(DATAFOLDER)
files = get_file_names(DATAFOLDER)

In [ ]:
files

['bnatt350.mps',
 'iis-bupa-cov.mps',
 'iis-pima-cov.mps',
 'acc-tight5.mps',
 'beasleyC3.mps',
 'bienst2.mps',
 '30n20b8.mps',
 'aflow40b.mps',
 '.ipynb_checkpoints']

In [ ]:
mps_lists = []

for file in [files[0]]:
  file = "bienst2.mps"
  print("Will start to load MPS-file " + file)

  try:
    tik = time()
    mps_list = load_mps(DATAFOLDER + file)
    tok = time()
    print(f"Loading {file} took {tok - tik} seconds")
    mps_lists.append(mps_list)

  except Exception as e:
    print(e)
    print(f"Could not find file {file}, on to the next")
    continue

Will start to load MPS-file bienst2.mps
Loading bienst2.mps took 0.3227815628051758 seconds


In [ ]:
cvxpy_times = []
ls_gpu_times = []
ls_cpu_times = []
used_n_values = []

for mps_list in mps_lists:

  c, A, b, C, d, up_bnd, lo_bnd, n = parse_mps_list(mps_list)

  print(f"Dimension n is: {n}")
  used_n_values.append(n)

  if up_bnd is not None:
    up_mask_inf = (up_bnd == np.inf)
    up_mask = (up_bnd != np.inf)
    up_bnd[up_mask_inf] = 1e9
  if lo_bnd is not None:
    lo_mask_inf = (lo_bnd == -np.inf)
    lo_mask = (lo_bnd != -np.inf)
    lo_bnd[lo_mask_inf] = -1e9

  print("Solve in CVXPY")
  total_cvxpy_time = 0
  for k in range(3):

    # Solve in CVXPY
    x = cp.Variable(n)

    # Objective
    obj = c @ x

    # Constraints
    constr = []

    if A is not None:
      constr.append(A @ x == b)
    if C is not None:
      constr.append(C @ x <= d)
    if lo_bnd is not None:
      constr.append(x >= lo_bnd)
    if up_bnd is not None:
      constr.append(x <= up_bnd)

    # Create problem
    prob = cp.Problem(cp.Minimize(obj), constr)

    # Solve
    tik = time()
    prob.solve(solver = cp.CLARABEL)
    print(f"CVXPY solved {k} time(s)")
    tok = time()

    total_cvxpy_time += tok - tik

    print(f"Problem is {prob.status}")

    del x
    del constr
    del prob

  cvxpy_time = (total_cvxpy_time) / 3
  cvxpy_times.append(cvxpy_time)

  print("LP-solver, GPU")
  total_ls_gpu_time = 0

  for k in range(3):
    ls_gpu = LPSolver(
        c = c,
        A = A,
        b = b,
        C = C,
        d = d,
        lower_bound = lo_bnd,
        upper_bound = up_bnd,
        use_gpu = True,
        suppress_print = True
    )

    tik = time()
    ls_gpu.solve()
    print(f"LP-solver, GPU solved {k} time(s)")
    tok = time()
    total_ls_gpu_time += tok - tik
    del ls_gpu

  ls_gpu_time = total_ls_gpu_time / 3
  ls_gpu_times.append(ls_gpu_time)

  print("LP-solver, CPU")
  total_ls_cpu_time = 0

  for k in range(3):
    ls_cpu = LPSolver(
        c = c,
        A = A,
        b = b,
        C = C,
        d = d,
        lower_bound = lo_bnd,
        upper_bound = up_bnd,
        use_gpu = False,
        suppress_print = True
    )

    tik = time()
    ls_cpu.solve()
    print(f"LP-solver, CPU solved {k} time(s)")
    tok = time()
    total_ls_cpu_time += tok - tik
    del ls_cpu

  ls_cpu_time = total_ls_cpu_time / 3
  ls_cpu_times.append(ls_cpu_time)

  print(f"Results for file {file}")
  print(f"CVXPY average time: {cvxpy_time}")
  print(f"LS GPU average time: {ls_gpu_time}")
  print(f"LS CPU average time: {ls_cpu_time}")
  print("\n")

Dimension n is: 505
Solve in CVXPY
CVXPY solved 0 time(s)
Problem is optimal
CVXPY solved 1 time(s)
Problem is optimal
CVXPY solved 2 time(s)
Problem is optimal
LP-solver, GPU
LP-solver, GPU solved 0 time(s)
LP-solver, GPU solved 1 time(s)
LP-solver, GPU solved 2 time(s)
LP-solver, CPU
LP-solver, CPU solved 0 time(s)
LP-solver, CPU solved 1 time(s)
LP-solver, CPU solved 2 time(s)
Results for file bienst2.mps
CVXPY average time: 0.02830163637797038
LS GPU average time: 14.007461706797281
LS CPU average time: 9.081709941228231




In [ ]:
# Blown out of the water by CVXPY. Sparse matrix kills it

In [ ]:
# Generate random, dense data and compare
np.random.seed(1)
n_values = [100, 500, 1000, 2000, 3000]
m_values = [80, 400, 800, 1600, 2400]
k_values = [20, 100, 200, 400, 600]

cvxpy_times_random = []
ls_gpu_times_random = []
ls_cpu_times_random = []

for n, m, k in zip(n_values, m_values, k_values):
  print(f"n is {n}")
  print("Generate some data")
  # Generate A
  A = np.random.uniform(low = -2, high = 2, size = (m, n))

  # Generate C
  C = np.random.uniform(low = -2, high = 2, size = (k, n))

  # Generate x_feas and c
  x_feas = np.random.uniform(low = -2, high = 2, size = (n))
  c = np.random.uniform(low = -2, high = 2, size = (n))

  # From this, calculate b and d
  b = A @ x_feas
  d = C @ x_feas

  # Have upper and lower bounds
  up_bnd = 3
  lo_bnd = -3

  # Create CVXPY problem
  print("Solve in CVXPY")
  total_cvxpy_time = 0
  for k in range(3):

    # Solve in CVXPY
    x = cp.Variable(n)

    # Objective
    obj = c @ x

    # Constraints
    constr = [A @ x == b, C @ x <= d, lo_bnd <= x, x <= up_bnd]

    # Create problem
    prob = cp.Problem(cp.Minimize(obj), constr)

    # Solve
    tik = time()
    prob.solve(solver = cp.CLARABEL)
    print(f"CVXPY solved {k} time(s)")
    tok = time()

    total_cvxpy_time += tok - tik


    del x
    del constr
    del prob

  print(f"Problem is {prob.status}")
  print(f"CVXPY gets optimal value of {obj.value}")

  cvxpy_time = total_cvxpy_time / 3
  cvxpy_times_random.append(cvxpy_time)

  print("LP-solver, GPU")
  total_ls_gpu_time = 0

  for k in range(3):
    ls_gpu = LPSolver(
        c = c,
        A = A,
        b = b,
        C = C,
        d = d,
        lower_bound = lo_bnd,
        upper_bound = up_bnd,
        use_gpu = True,
        suppress_print = True
    )

    tik = time()
    ls_gpu.solve()
    print(f"LP-solver, GPU solved {k} time(s)")
    tok = time()
    total_ls_gpu_time += tok - tik
    del ls_gpu

  ls_gpu_time = total_ls_gpu_time / 3
  ls_gpu_times_random.append(ls_gpu_time)

  print("LP-solver, CPU")
  total_ls_cpu_time = 0

  for k in range(3):
    ls_cpu = LPSolver(
        c = c,
        A = A,
        b = b,
        C = C,
        d = d,
        lower_bound = lo_bnd,
        upper_bound = up_bnd,
        use_gpu = False,
        suppress_print = True
    )

    tik = time()
    ls_cpu.solve()
    print(f"LP-solver, CPU solved {k} time(s)")
    tok = time()
    total_ls_cpu_time += tok - tik
    del ls_cpu

  ls_cpu_time = total_ls_cpu_time / 3
  ls_cpu_times_random.append(ls_cpu_time)

  print(f"Results for n = {n}")
  print(f"CVXPY average time: {cvxpy_time}")
  print(f"LS GPU average time: {ls_gpu_time}")
  print(f"LS CPU average time: {ls_cpu_time}")
  print("\n")

n is 100
Generate some data
Solve in CVXPY
CVXPY solved 0 time(s)
Problem is optimal
CVXPY solved 1 time(s)
Problem is optimal
CVXPY solved 2 time(s)
Problem is optimal
LP-solver, GPU
LP-solver, GPU solved 0 time(s)
LP-solver, GPU solved 1 time(s)
LP-solver, GPU solved 2 time(s)
LP-solver, CPU
LP-solver, CPU solved 0 time(s)
LP-solver, CPU solved 1 time(s)
LP-solver, CPU solved 2 time(s)
Results for n = 100
CVXPY average time: 0.03304704030354818
LS GPU average time: 2.447383483250936
LS CPU average time: 0.3091271718343099


n is 500
Generate some data
Solve in CVXPY
CVXPY solved 0 time(s)
Problem is optimal
CVXPY solved 1 time(s)
Problem is optimal
CVXPY solved 2 time(s)
Problem is optimal
LP-solver, GPU
LP-solver, GPU solved 0 time(s)
LP-solver, GPU solved 1 time(s)
LP-solver, GPU solved 2 time(s)
LP-solver, CPU
LP-solver, CPU solved 0 time(s)
LP-solver, CPU solved 1 time(s)
LP-solver, CPU solved 2 time(s)
Results for n = 500
CVXPY average time: 2.3835979302724204
LS GPU average tim

In [ ]:
print(cvxpy_times_random)
print(ls_gpu_times_random)
print(ls_cpu_times_random)

[0.03304704030354818, 2.3835979302724204, 16.64044197400411, 232.35090923309326]
[2.447383483250936, 5.03038493792216, 17.538308540980022, 91.77346483866374]
[0.3091271718343099, 3.8650670051574707, 25.577720085779827, 191.94537933667502]


In [ ]:
# T4 GPU
# cvxpy_times_random = [0.03304704030354818, 2.3835979302724204, 16.64044197400411, 232.35090923309326]
# ls_gpu_times_random = [2.447383483250936, 5.03038493792216, 17.538308540980022, 91.77346483866374]
# ls_cpu_times_random = [0.3091271718343099, 3.8650670051574707, 25.577720085779827, 191.94537933667502]